# Preprocessing Statkraft and Cerra data

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import xarray as xr

import datetime
import pandas as pd
from shapely.geometry import Point

import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from pyproj import CRS, Transformer



In [ ]:
# Load the shapefile
shp_file = "catchment_statkraft/catchment.shp"
catchment = gpd.read_file(shp_file)
# Check the coordinate reference system (CRS)
print(catchment.crs)
# Plot the shapefile
catchment.plot();

In [ ]:
# Open the NetCDF file
nc_file = "atnasjo_snow_model.nc"
ds = xr.open_dataset(nc_file)
# Print dataset metadata
print(ds.variables)

In [ ]:
# Select time slice (e.g., first time step)
ds.sel(time='2015-01-01')['sde'].plot()
plt.show()

In [ ]:
# Extract and Assign CRS from NetCDF
if 'crs_wkt' in ds['projection_lambert'].attrs:  # Adjust 'projection_lambert' if needed
    netcdf_crs = CRS.from_wkt(ds['projection_lambert'].attrs['crs_wkt'])
    print("Extracted CRS:", netcdf_crs)
else:
    netcdf_crs = None
    print("No CRS found in NetCDF file!")
    
# Reproject Catchment Shapefile to NetCDF CRS
if catchment.crs != netcdf_crs:
    catchment = catchment.to_crs(netcdf_crs)

In [ ]:
# Extract Data
sde = ds['sde'].isel(time=0)  # First time step
x = ds['x'].values  # X-coordinates (projected)
y = ds['y'].values  # Y-coordinates (projected)



# 🔹 Fix Grid Cell Alignment by Creating an Edged Grid
X, Y = np.meshgrid(
    np.linspace(x.min(), x.max(), sde.shape[1] + 1),  # X should be 1 larger than sde.shape[1]
    np.linspace(y.min(), y.max(), sde.shape[0] + 1)   # Y should be 1 larger than sde.shape[0]
)

# 🔹 Plot the Data Using pcolormesh()
fig, ax = plt.subplots(figsize=(8, 6))

c = ax.pcolormesh(X, Y, sde.values, cmap="Blues", shading="flat")  # Corrected!

# Add a Colorbar
cb = plt.colorbar(c, ax=ax)
cb.set_label("Snow Depth (m)")


# Overlay Catchment Shapefile (Correctly Reprojected)
catchment.boundary.plot(ax=ax, edgecolor="red", linewidth=2, label="Catchment Boundary")
# Formatting
ax.set_xlabel("X Coordinate (meters)")
ax.set_ylabel("Y Coordinate (meters)")
ax.set_title("Corrected Snow Depth Map with Catchment Overlay")
ax.set_aspect('equal')
plt.legend()

plt.show()


### Plot of the catchment with points, crs from .nc file converted

In [ ]:


# Extract Data
sde = ds['sde'].isel(time=0)  # First time step
x = ds['x'].values  # X-coordinates (projected)
y = ds['y'].values  # Y-coordinates (projected)

# Create a meshgrid of X and Y
X, Y = np.meshgrid(x, y)  # Convert 1D x and y arrays to a full grid

# Flatten the arrays for creating GeoDataFrame
X_flat = X.ravel()
Y_flat = Y.ravel()

# Create a GeoDataFrame of Grid Points
grid_points = gpd.GeoDataFrame(
    {'x': X_flat, 'y': Y_flat},
    geometry=[Point(x, y) for x, y in zip(X_flat, Y_flat)],
    crs=netcdf_crs
)

# Select Grid Points Inside the Catchment
points_within = grid_points[grid_points.geometry.within(catchment.geometry.union_all())]

# Print the points within the catchment
print("Points within the catchment:")
print(points_within)

# Plot the Data Using pcolormesh()
fig, ax = plt.subplots(figsize=(8, 6))

# Create an edged grid for pcolormesh
X_edge, Y_edge = np.meshgrid(
    np.linspace(x.min(), x.max(), sde.shape[1] + 1),  # X should be 1 larger than sde.shape[1]
    np.linspace(y.min(), y.max(), sde.shape[0] + 1)   # Y should be 1 larger than sde.shape[0]
)
c = ax.pcolormesh(X_edge, Y_edge, sde.values, cmap="viridis", shading="flat")

# Add a Colorbar
cb = plt.colorbar(c, ax=ax)
cb.set_label("Snow Depth (m)")

# Overlay Catchment Shapefile (Correctly Reprojected)
catchment.boundary.plot(ax=ax, edgecolor="black", linewidth=2, label="Catchment Boundary")

# Plot the points within the catchment
points_within.plot(ax=ax, 
                   color='red', 
                   markersize=20, 
                   label="Points within Catchment")

# Formatting
ax.set_xlabel("X Coordinate (meters)")
ax.set_ylabel("Y Coordinate (meters)")
ax.set_title("Snow Depth Map with Catchment Overlay and Points within Catchment")
ax.set_aspect('equal')
plt.legend()

plt.show()


In [ ]:
sde = ds['sde'] 
time_series_data = []
for _, row in points_within.iterrows():
    x_pt, y_pt = row['x'], row['y']
    idx_x = (np.abs(x - x_pt)).argmin()
    idx_y = (np.abs(y - y_pt)).argmin()
    sde_values = sde[:, idx_y, idx_x].values
    for t, sde_value in zip(ds['time'].values, sde_values):
        time_series_data.append({
            'time': pd.Timestamp(t),
            'x': x_pt,
            'y': y_pt,
            'statkraft_sde': sde_value
        })
# Create a DataFrame with the Time Series Data
statkraft_sde_df = pd.DataFrame(time_series_data)

In [ ]:
# Identify unique (x, y) pairs
unique_xy = statkraft_sde_df[['x', 'y']].drop_duplicates().reset_index(drop=True)

# Assign a unique index to each unique (x, y) pair
unique_xy['unit'] = range(len(unique_xy))

# Merge the unique indices back to the original DataFrame
statkraft_sde_df = statkraft_sde_df.merge(unique_xy, on=['x', 'y'], how='left')
statkraft_sde_df.set_index('time', inplace=True)

# Display the updated DataFrame
print("Updated DataFrame with 'unit' column:")
print(statkraft_sde_df.head())

In [ ]:
statkraft_sde_df.unit.unique()

In [ ]:
# Pivot the DataFrame to have units as columns
pivot_statkraft = statkraft_sde_df.pivot(columns='unit', 
                                         values='statkraft_sde')

# Plot the time series for each unit with subplots
pivot_statkraft.plot(subplots=True, 
                     sharex=True,
                     #sharey=True, 
                     figsize=(15, 1 * len(pivot_statkraft.columns)), 
                     layout=(len(pivot_statkraft.columns), 1), 
                     legend=False)

# Set the x-axis label for the last subplot
plt.xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Create a transformer from netcdf_crs to EPSG:4326 (WGS84 lat/lon)
transformer = Transformer.from_crs(netcdf_crs, "EPSG:4326", always_xy=True)

# Convert the x and y coordinates to longitude and latitude using .loc to avoid SettingWithCopyWarning
points_within.loc[:, "longitude"], points_within.loc[:, "latitude"] = transformer.transform(
    points_within["x"].values, points_within["y"].values
)
print("Converted UTM to Lat/Lon:")
print(points_within[['x', 'y', 'longitude', 'latitude']].head())

### Run to retrieve CERRA data from scratch

In [ ]:

# Import the Open-Meteo API client from openmeteo_requests

import openmeteo_requests
import requests_cache
import numpy as np
import pandas as pd
from retry_requests import retry

# Setup Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Define API URL
url = "https://archive-api.open-meteo.com/v1/archive"

# Initialize an empty list to store time-series results
cerra_time_series_data = []

# Define the start and end date for the time series (Match Statkraft!)
start_date = "2015-01-01"
end_date = "2020-03-01"

# Loop through each grid point and get a full time series
for index, row in points_within.iterrows():
    params = {
        "latitude": row["latitude"],
        "longitude": row["longitude"],
        "start_date": start_date,
        "end_date": end_date,
        "hourly": "snow_depth",  # Fetching Hourly
        "cell_selection": "nearest",
        "timezone": "UTC"
    }

    try:
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]

        # Extract hourly snow depth
        hourly = response.Hourly()
        dates = pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        )
        cerra_snow_depth = hourly.Variables(0).ValuesAsNumpy()

        # Store results for the full time series
        for i in range(len(dates)):
            cerra_time_series_data.append({
                "x": row["x"],
                "y": row["y"],
                "longitude": row["longitude"],
                "latitude": row["latitude"],
                "date": dates[i].date(),  # Convert to daily format
                "hourly_snow_depth": cerra_snow_depth[i]
            })

        print(f"Retrieved full time series for CERRA at ({row['latitude']:.4f}, {row['longitude']:.4f})")

    except Exception as e:
        print(f"Failed for {row['latitude']:.4f}, {row['longitude']:.4f}: {e}")

# Convert results to DataFrame
cerra_hourly_df = pd.DataFrame(cerra_time_series_data)

# Convert hourly data to daily averages
cerra_daily_df = cerra_hourly_df.groupby(["x", "y", "longitude", "latitude", "date"])["hourly_snow_depth"].mean().reset_index()

# Rename for clarity
cerra_daily_df.rename(columns={"hourly_snow_depth": "cerra_snow_depth"}, inplace=True)

# Display Retrieved Daily Time-Series Data
print("Successfully Converted CERRA Data to Daily Resolution")
print(cerra_daily_df.head())

cerra_daily_df.to_csv("cerra_snow_depth_new.csv", index=False)


### Open CERRA data already fetched

In [ ]:
cerra_sde_df = pd.read_csv("cerra_snow_depth_new.csv")

In [ ]:
# Rename the 'date' column to 'time' and convert it to datetime
cerra_sde_df.rename(columns={'date': 'time'}, inplace=True)
cerra_sde_df.rename(columns={'cerra_snow_depth': 'cerra_sde'}, inplace=True)
cerra_sde_df['time'] = pd.to_datetime(cerra_sde_df['time'])


# Display the updated DataFrame
print("Updated DataFrame with 'time' column as datetime index:")
print(cerra_sde_df.head())


In [ ]:
# Merge the unique indices back to the original DataFrame
cerra_sde_df = cerra_sde_df.merge(unique_xy, on=['x', 'y'], how='left')
cerra_sde_df.set_index('time', inplace=True)

# Display the updated DataFrame
print("Updated DataFrame with 'unit' column:")
print(cerra_sde_df.head())



In [ ]:
# Pivot the DataFrame to have units as columns
pivot_cerra = cerra_sde_df.pivot_table(index='time', columns='unit', values='cerra_sde')


# Plot the time series for each unit with subplots
pivot_cerra.plot(subplots=True, 
                sharex=True, 
                sharey = True,
                figsize=(15, 1 * len(pivot_cerra.columns)), 
                layout=(len(pivot_cerra.columns), 1), 
                legend=False)

# Set the x-axis label for the last subplot
plt.xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
# Plot the time series for each unit with subplots
fig, axes = plt.subplots(nrows=len(pivot_cerra.columns), ncols=1, sharex=True, figsize=(15, 1 * len(pivot_cerra.columns)))

for i, unit in enumerate(pivot_cerra.columns):
    ax = axes[i]
    pivot_cerra[unit].plot(ax=ax, label='CERRA', color='blue')
    pivot_statkraft[unit].plot(ax=ax, label='Statkraft', color='orange')
    
    ax.set_title(f'Unit {unit}')
    ax.legend()

# Set the x-axis label for the last subplot
axes[-1].set_xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()

### Check to see whether the CERRA data is different wrt units

In [ ]:
# Pivot the DataFrame to have units as columns
pivot_cerra = cerra_sde_df.pivot_table(index='time', columns='unit', values='cerra_sde')


# Plot the time series for each unit with subplots
pivot_cerra.plot(subplots=False, 
                sharex=True, 
                sharey = True,
                figsize=(12, 3), 
                layout=(len(pivot_cerra.columns), 1), 
                legend=False)

# Set the x-axis label for the last subplot
plt.xlabel('Time')

# Adjust layout
plt.tight_layout()

# Display the plot
plt.show()